# StarGAN : Unified Generative Adversarial Networks for Multi-Domain Image-to-Image Translation

## Abstract

- single generator로 multi domain에 대한 image to image translation
- 서로 다른 도메인에 속한 데이터셋을 같이 학습 가능(joint training)

<img src='image/1.PNG', align='left'>

## 1. Introduction

### Image-to-image translation

- 주어진 이미지에서 특정 부분을 다르게 변형시키는 것
- GAN 등장 이후로 빠르게 발전, 머리 색 변경/지도 변형/풍경 이미지에서 계절 변경 등 가능
- 일반적으로는 두 개의 서로 다른 도메인의 학습 데이터를 이용하여, 하나의 도메인에서 다른 도메인으로 이미지를 변형함
---
- __attribute__ : 이미지가 가지는 의미있는 특성 (머리 색깔, 성별, 나이 등)
- __attribute value__ : attribute의 특정한 값 (머리 색깔 -> black/brown/blond, 성별 -> male/female, 나이 -> young/old)
- __domain__ : 같은 attribute value를 공유하는 이미지 셋 (domain1 -> female, domain2 -> male)

### Cross-domain models vs. StarGAN

- 기존 image-to-image translation의 경우, k개의 도메인이 있으면 K(k-1)개의 generator를 학습해야 함
- 얼굴 형태와 같이 모든 도메인에서 학습가능한 global feature가 존재하긴 하지만 각 generator가 전체 학습 데이터를 충분히 활용하지 못하고 k개 중에 두 개 도메인에 대해서만 학습이 가능
- 학습 데이터를 충분히 활용하지 못하면 생성되는 이미지 품질이 저하됨

<img src='image/2.PNG'>

- StarGAN은 여러 도메인을 하나의 네트워크로 학습 가능
- input으로 이미지와 도메인 정보를 함께 넣어서, 이미지가 대응되는 도메인으로 분류(변형)되도록 학습함
- 도메인 정보를 표현하기 위해서 one-hot vector로 표현한 label을 사용
- 학습할 때, target 도메인 label을 랜덤하게 생성하고, 이미지를 그에 맞게 변형
- 도메인 label에 mask vector를 추가하여 서로 다른 데이터셋의 도메인들을 같이 학습 가능

### Datasets

#### 1) CelebA
- The CelebFaces Attributes (CelebA) dataset
- celebrities의 얼굴 사진 202,599개, 각 사진은 40개의 binary attributes가 표기되어 있음
- 178×218(원본) -> 178×178로 크롭 -> 128×128로 resize
- test set으로 랜덤하게 2,000개 추출/나머지는 학습에 사용
- 다음 attribute로 총 7개의 도메인 설정
    - hair color (black, blond, brown), gender (male/female), and age (young/old)
    
#### 2) RaFD
- The Radboud Faces Database (RaFD) dataset
- 67명의 참가자로부터 수집한 4,824개 이미지
- 8가지 표정 x 3개 시선 x 3개 각도
- 256×256 크롭 -> 128×128로 resize
---
- multi-domain뿐만 아니라, multi-dataset에 대해서 한번에 학습("joint training")
- RaFD을 학습하여 얻은 feature들을 사용하여 CelebA 이미지의 facial expression을 변경(facial attribute transfer + facial expression synthesis)

## 2. Related Work

- GAN
- Conditional GAN
- Image-to-Image Translation
    - pix2pix
    - UNIT
    - CoGAN
    - CycleGAN
    - DiscoGAN

## 3. StarGAN

- multiple domain을 mapping하는 single generator G를 학습하는 것이 목표

> __G(x,c) → y__

> x: input image, c: target domain label, y: output image

- input으로 이미지와 도메인 정보를 함께 넣어서 이미지가 대응되는 도메인으로 분류(변형)되도록 학습함
- target domain label c를 랜덤하게 생성하여, G가 input image를 flexible하게 학습할 수 있도록 함
- single discriminator D가 multiple domain을 다룰 수 있도록 하는 보조 classifier(auxiliary classifier) 사용. 즉, discriminator D는 source와 domain label 모두에 대한 확률분포를 생성

> __D : x → {Dsrc(x), Dcls(x)}__

<img src='image/4.PNG', align='left'>

### 3.1. Objective Function

### 1) Adversarial Loss
<img src='image/eq1.PNG', align='left'>

- Dsrc(x) : source(Real/Fake)에 대한 확률분포
- __disciminator D__ tries to maximize \\(L_{adv}\\). real image x가 들어오면 확률이 높아야 하고, fake image G(x,c)가 들어오면 확률이 낮아야 함. real/fake image를 잘 구분하도록 학습됨
- __generator G__ tries to minimize \\(L_{adv}\\). fake image G(x,c)가 들어왔을 때 확률이 높아야 함.

### 2) Domain Classification Loss
- input image x와 target domain label c가 있을 때, x를 -> 도메인 c에 속하는 output image y로 변형하는 것이 목표
- 이를 위해 보조 classifier를 추가하여 D와 G를 학습할 때 domain classification loss를 고려하도록 함
- 목적함수를 두 가지로 구성
    - domain classification loss of real images used to optimize D
    - domain classification loss of fake images used to optimize G
- Dcls(x) : domain에 대한 확률분포

<img src='image/eq2.PNG', align='left'>
.
- real image에 대한 domain classification loss
- c' : real image x에 대응되는 original domain label
- __disciminator D__ tries to minimize \\(L^{r}_{cls}\\). real image x를 대응되는 original domain c'로 분류하는 방향으로 학습함

<img src='image/eq3.PNG', align='left'>
.
- fake image에 대한 domain classification loss
- __generator G__ tries to minimize \\(L^{f}_{cls}\\). fake image G(x,c)를 target domain c로 분류하는 방향으로 학습함

### 3) Reconstruction Loss
- generator G는 \\(L_{adv}\\)와 \\(L^f_{cls}\\)를 minimize함으로써 1)real image에 가까우며 2)target domain에 맞는 이미지를 생성함
- 하지만, 변형된 이미지가 input image의 특성을 잘 보존하고 있는지는 확신할 수 없으므로, 해당 loss를 추가함
- cycleGAN에서 사용하는 cycle consistency loss와 동일

<img src='image/eq4.PNG', align='left'>
.
- input image: G가 생성한 fake image, domain: original domain label
- __generator G__ tries to minimize \\(L_{rec}\\).
- L1 norm 사용
- generator G는 fake image를 생성할 때/생성한 fake image를 original image로 reconstruct할 때 총 두번 학습됨
---
위의 loss 함수들을 종합하면, 전체 목적함수 형태는 다음과 같다.

<img src='image/eq56.PNG', align='left'>

- \\(\lambda_{cls}\\)와 \\(\lambda_{rec}\\)는 domain classification loss와 reconstriction loss의 중요도를 조절하는 hyperparameter
- 논문에서는 \\(\lambda_{cls}\\)=1, \\(\lambda_{rec}\\)=10 사용

### 3.2. Mask Vector

- StarGAN은 서로 다른 유형의 label을 포함한 여러 데이터셋을 동시에 학습하여 test 시점에 모든 label을 control 가능
- 하지만 여러 데이터셋을 동시에 학습할 때 문제는 label 정보가 데이터셋별로 각각 존재(partially known to each dataset)한다는 것
- CelebA와 RaFD의 경우를 예로 들면, CelebA는 머리색/성별 등에 대한 label 정보는 포함하지만 happy/angry 등에 대한 label 정보는 포함하지 않는다.
- G가 생성한 이미지 G(x,c)를 가지고 input image x를 reconstruct할 때 original domain label c'가 필요하므로, 위와 같은 점은 문제가 됨
- 이런 문제를 해결하기 위해 Mask Vector m을 도입하여 모르는(unspecified) label은 무시하고 알고 있는 label에만 집중하도록 함
- StarGAN은 m을 표현하기 위해 n-dimensional one-hot vector를 사용함(n: dataset 개수, n=2)
- unified version of the label as a vector

    <img src='image/eq7.PNG', align='left'>
    
.
- ci represents a vector for the labels of the i-th dataset. The vector of the known label ci can be represented as either a binary vector for binary attributes or a one-hot vector for categorical attributes. For the remaining n−1 unknown labels we simply assign zero values. 

## 4. Training

- 학습 과정을 안정화하고 이미지 품질을 좋게 하기 위해서 \\(L_{adv}\\)를 다음과 같이 변경함 (gradient penalty가 있는 Wasserstein GAN의 목적함수 차용)
    - \\(\hat{x}\\) is sampled uniformly along a straight line between a pair of a real and a generated images
    - \\(\lambda_{gp}\\)=10 사용
    
    <img src='image/eq8.PNG', align='left'>

- generator network
    - 2 convolution layers with the stride size of 2 for downsampling
    - 6 residual blocks
    - 2 transposed convolution layers with the stride size of 2 for upsampling (https://zzsza.github.io/data/2018/02/23/introduction-convolution/)
- instance normalization for the generator, no normalization for the discriminator (Batch normalization normalizes all images across the batch and spatial locations (in the ordinary case, in CNN it's different); instance normalization normalizes each batch independently, i.e., across spatial locations only)
- PatchGANs for the discriminator network(local image patch가 real인지 fake인지 구분하는 역할)
- trained using Adam(β1 = 0.5 and β2 = 0.999)
- horizontal flip
- one generator update after five discriminator updates
- batch size : 16
- CelebA) learning rate of 0.0001 for the first 10 epochs and linearly decay the learning rate to 0 over the next 10 epochs. 
- RaFD) To compensate for the lack of data, first 100 epochs with a learning rate of 0.0001 and apply the same decaying strategy over the next 100 epochs. 
- Training takes about one day on a single NVIDIA Tesla M40 GPU

<img src='image/3.PNG'>

## 5. Result

- 비교 모델
    - DIAT
    - CycleGAN
    - IcGAN
- Amazon Mechanical Turk(AMT)로 성능 평가 및 비교
- StarGAN이 다른 모델에 비해 더 높은 점수를 얻음

### Effects of joint training
<img src='image/5.PNG'>
- StarGAN-SNG : RaFD 데이터로만 학습 -> generates reasonable but blurry images with gray backgrounds
- StarGAN-JNT : mask vector를 사용하여 CelebA와 RaFD를 동시에 학습 -> high visual quality
- This difference is due to the fact that StarGAN-JNT learns to translate CelebA images during training but not StarGAN-SNG. In other words, StarGAN-JNT can leverage both datasets to improve shared low-level tasks such facial keypoint detection and segmentation. By utilizing both CelebA and RaFD, StarGAN-JNT can improve these low-level tasks, which is beneficial to learning facial expression synthesis.

### Learned Role of Mask vector
<img src='image/6.PNG'>
- celebA 데이터에 대해서 facial expression을 변경하고 싶은 경우 mask vector는 [0, 1]로 셋팅해야 함
- 실험 이미지 : young 특징을 지닌 celebA 이미지
- 첫 행의 이미지는 mask vector를 제대로 설정했을 때, 두번째 행의 이미지는 [1, 0]으로 잘못 설정했을 때의 결과
- facial expression에 대한 값이 0으로 들어가 있으므로 facial expression이 아닌 facial attribute에 집중하게 되고, 기존 attribute가 young이었으므로 old 이미지로 변형하게 됨

## 6. Conclusion

- single model로 여러 도메인/dataset에 대해 이미지 변형을 수행
- 기존 방법들(cycleGAN 등)보다 higher quality 이미지를 생성
- mask vector 개념을 도입하여 부분적으로 label된 여러 데이터셋을 사용 및 모든 가능한 도메인 label을 컨트롤 가능
- 도메인 종류에 상관없이 적용 가능

## Reference
- https://arxiv.org/pdf/1711.09020.pdf (논문)
- https://github.com/yunjey/StarGAN/blob/master/solver.py (코드)